Goal: pull out 1 buoy record that has turbidity and search for matching icesat2 granules

Order of operations:
- load info for 1 relevant station
- Find all matching atl granules and save times from file names
- Compare times to see if any matching buoy times


In [2]:
%pip install erddapy

  Using cached erddapy-2.2.0-py3-none-any.whl.metadata (6.6 kB)
Using cached erddapy-2.2.0-py3-none-any.whl (22 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta
from sliderule import sliderule, icesat2, earthdata

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def fname2datetime(fname):
    y = int(fname[6:10])
    m = int(fname[10:12])
    d = int(fname[12:14])
    H = int(fname[14:16])
    M = int(fname[16:18])
    S = int(fname[18:20])

    t = datetime(y,m,d,H,M,S, tzinfo=timezone.utc)
    return t
def buoy_bound_box(lat,lon):
    # define a buffer distance around the buoy to search for icesat-2 data
    buffer_km = 0.1 # buffer distance in km
    lat_buff = buffer_km/111 # convert buffer distance to frac of 1 deg lat
    lon_buff = buffer_km/(111*np.cos(lat*np.pi/180)) # convert buffer distance to frac of 1 deg lon
    # define bounding box around the buoy (WSEN)
    # example: bbox = [-108.3, 39.2, -107.8, 38.8]
    bbox = [lon-lon_buff,lat+lat_buff,lon+lon_buff,lat-lat_buff]
    region = sliderule.toregion(bbox)
    return region

In [6]:
# load a list of all the relevant ERDDAPs and their urls.
FF = pd.read_pickle("labeled_relevant_stations.pkl")
# only look at buoys with both ssc and chlorophyll
FF = FF[(FF.buoy == True) & (FF.ssc == True) & (FF.chl == True)]
FF.reset_index(drop=True, inplace=True)
l = len(FF)

In [7]:
# loop through each buoy asset:
for jj in range(3):
    # define a search region around the buoy 
    lat = FF['geospatial_lat_min'][jj]
    lon = FF['geospatial_lon_min'][jj]
    region = buoy_bound_box(lat,lon) 
    
    # search CMR for ATL03 granules in the bounding box
    grns = earthdata.cmr(
        short_name="ATL06",
        polygon=region['poly'],
        version='006')
    # save the times for each granule as a datetime object
    icesat_times = [fname2datetime(fname) for fname in grns]
    
    # now check if buoy data exists for these granules
    e = ERDDAP(server=FF['url'][jj],
               protocol="tabledap",
               response="csv")
    e.dataset_id = FF['sites'][jj]
    
    for t in icesat_times:
        # add a time buffer (+/-2 hours) to search for relevant buoy data for each granule
        t_start = (t-timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%S+00:00")
        t_end = (t+timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%S+00:00")
        e.constraints = {"time>=": t_start,
                         "time<=": t_end}
    
        # try to download the associated buoy data
        try: 
            buoy = e.to_pandas(parse_dates=True)
        except:
            print('no buoy data for this granule')
            print('')
            continue
        # if buoy data exists, download the ATL03 photons in the bounding box at this time
        print('downloading ATL03')
        print('')
        # # Build ATL03 Request
        # parms = {
        #     "poly": region['poly'],
        #     "t0": t_start,
        #     "t1": t_end, #'2023-01-22T00:00:00Z',
        #     "srt": icesat2.SRT_OCEAN,
        #     "len": 50.0,
        #     "res": 50.0
        # }
        # ATL03_gdf = icesat2.atl03sp(parms)

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

downloading ATL03

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

downloading ATL03

downloading ATL03

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

downloading ATL03

no buoy data for this granule

no buoy data for this granule

no buoy data for this granule

downloading ATL03

downloading ATL03

no buoy data for this granule

no buoy data for this granule

no buoy data for t